<a href="https://colab.research.google.com/github/yuvalofek/sqlite_test/blob/main/Learning_SQLite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Testing out sqlite and sqlite3 

In [ ]:
import sqlite3
from pprint import pprint

In [ ]:
# make a connection with the db
conn = sqlite3.connect('learning_sql.db')
c = conn.cursor()

In [ ]:
def print_table(table, cursor):
  """
  Handy function to quickly print a table (not ideal bc of SQL injection attacks,
  but since I am just reading I let it go)
  table (str): table name
  """

  cmd = """SELECT * FROM {};
    """.format(table)
  cursor.execute(cmd)

  pprint(c.fetchall())

In [ ]:
# creating a table
c.execute("""
CREATE TABLE IF NOT EXISTS student (
  student_id INT PRIMARY KEY, 
  name TEXT, 
  major TEXT
);
""")
conn.commit()

# seeing what is in the table
print_table('student', c)


[]


In [ ]:
# inserting into table
c.execute(""" INSERT INTO student VALUES {};""".format((0, 'Bob Dylan', 'Mathematics')))
conn.commit()

# seeing what is in the table
print_table('student', c)

# insert many into table
c.executemany(""" INSERT INTO student VALUES (?,?,?); """, [(2, 'Jeremy Pierce', 'Robotics'), 
                                                   (1, 'James Ryan', 'Computer Science')])
conn.commit()


# seeing what is in the table
print_table('student', c)


[(0, 'Bob Dylan', 'Mathematics')]
[(0, 'Bob Dylan', 'Mathematics'),
 (2, 'Jeremy Pierce', 'Robotics'),
 (1, 'James Ryan', 'Computer Science')]


In [ ]:
# adding a  gpa column
c.execute("""ALTER TABLE student ADD gpa REAL;""")
conn.commit()

# seeing what is in the table
print_table('student', c)


[(0, 'Bob Dylan', 'Mathematics', None),
 (2, 'Jeremy Pierce', 'Robotics', None),
 (1, 'James Ryan', 'Computer Science', None)]


In [ ]:
# Update a  gpa to all student using WHERE and IS NULL
c.execute("""UPDATE student
          SET GPA = 2.5
          WHERE GPA IS NULL;""")
conn.commit()
print_table('student', c)

# update the gpa of student 0
c.execute("""UPDATE student
          SET GPA = 2.1
          WHERE student_id=0;""")
conn.commit()
print_table('student', c)

# LIKE 
c.execute("""UPDATE student
          SET major = 'Comp Sci'
          WHERE major LIKE '%Computer%'; """)
conn.commit()
print_table('student', c)


[(0, 'Bob Dylan', 'Mathematics', 2.5),
 (2, 'Jeremy Pierce', 'Robotics', 2.5),
 (1, 'James Ryan', 'Computer Science', 2.5)]
[(0, 'Bob Dylan', 'Mathematics', 2.1),
 (2, 'Jeremy Pierce', 'Robotics', 2.5),
 (1, 'James Ryan', 'Computer Science', 2.5)]
[(0, 'Bob Dylan', 'Mathematics', 2.1),
 (2, 'Jeremy Pierce', 'Robotics', 2.5),
 (1, 'James Ryan', 'Comp Sci', 2.5)]


In [ ]:
# select with order by
c.execute("""SELECT * FROM student ORDER BY major;""")
pprint(c.fetchall())
conn.commit()

# descending student_id
c.execute("""SELECT * FROM student ORDER BY student_id DESC;""")
pprint(c.fetchall())
conn.commit()

[(1, 'James Ryan', 'Comp Sci', 2.5),
 (0, 'Bob Dylan', 'Mathematics', 2.1),
 (2, 'Jeremy Pierce', 'Robotics', 2.5)]
[(2, 'Jeremy Pierce', 'Robotics', 2.5),
 (1, 'James Ryan', 'Comp Sci', 2.5),
 (0, 'Bob Dylan', 'Mathematics', 2.1)]


In [ ]:
# SELECT MIN
c.execute("""SELECT MIN(gpa) FROM student;""")
print(c.fetchall())

# ALIAS
c.execute("""SELECT MIN(gpa) FROM student AS LowestGPA;""")
print(c.fetchall())

# SELECT with aggregation
c.execute("""SELECT (name), MIN(gpa) FROM student;""")
print(c.fetchall())

# Count the students
c.execute("""SELECT count(student_id) FROM student;""")
print(c.fetchall())

# Average GPA
c.execute("""SELECT AVG(gpa) FROM student;""")
print(c.fetchall())


[(2.1,)]
[(2.1,)]
[('Bob Dylan', 2.1)]
[(3,)]
[(2.3666666666666667,)]


In [ ]:
# delete
c.execute("""DELETE FROM student WHERE gpa=2.1;""")

print_table('student', c)
conn.commit()

[(2, 'Jeremy Pierce', 'Robotics', 2.5), (1, 'James Ryan', 'Comp Sci', 2.5)]


In [ ]:
import pandas as pd
# to dataframe
conn = sqlite3.connect('learning_sql.db')

df = pd.read_sql_query('SELECT * from student;', conn, )
df.head()

conn = sqlite3.connect('learning_sql.db')
c = conn.cursor()
print_table('student', c )
conn.close()

[(2, 'Jeremy Pierce', 'Robotics', 2.5), (1, 'James Ryan', 'Comp Sci', 2.5)]


In [ ]:
# Add a none specialty column and save as table student2
conn = sqlite3.connect('learning_sql.db')
df['specialty']= None
df.to_sql('student2', conn, if_exists='replace')

# see new table
conn = sqlite3.connect('learning_sql.db')
c = conn.cursor()
print_table('student2', c)

# drop new table
c = conn.cursor()
c.execute("""DROP TABLE student2;""")
conn.commit()
conn.close()

[(0, 2, 'Jeremy Pierce', 'Robotics', 2.5, None),
 (1, 1, 'James Ryan', 'Comp Sci', 2.5, None)]


In [ ]:
# dropping existing student table
conn = sqlite3.connect('learning_sql.db')
c = conn.cursor()
c.execute("""DROP TABLE student;""")
conn.commit()

# trying to fetch from dropped table -> expecting an error
c.execute(""" SELECT * FROM student;""")

print(c.fetchall())
conn.commit()

OperationalError: ignored